# LangChain Agent with Qiskit IBM Runtime MCP Server

This notebook demonstrates how to create an AI agent using LangGraph that connects to the **qiskit-ibm-runtime-mcp-server** via the Model Context Protocol (MCP).

The agent can interact with IBM Quantum services to:
- List available quantum backends
- Find the least busy backend
- Get detailed backend properties and calibration data
- Manage quantum jobs

## Architecture

```
┌─────────────┐     MCP Protocol     ┌──────────────────────────────────┐
│  LangChain  │ ◄──────────────────► │ qiskit-ibm-runtime-mcp-server    │
│    Agent    │                      │                                  │
└─────────────┘                      │  ┌────────────────────────────┐  │
                                     │  │   qiskit-ibm-runtime       │  │
                                     │  └────────────────────────────┘  │
                                     │               │                  │
                                     └───────────────│──────────────────┘
                                                     ▼
                                            ┌─────────────────┐
                                            │  IBM Quantum    │
                                            │    Cloud        │
                                            └─────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server
pip install qiskit-ibm-runtime-mcp-server

# Install LangChain dependencies
pip install langgraph langchain-mcp-adapters python-dotenv

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your IBM Quantum token and (optionally) instance for faster startup.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

In [ ]:
import os
from dotenv import load_dotenv

# Load from .env file if it exists
load_dotenv()

# Or set directly (uncomment and fill in):
# os.environ["QISKIT_IBM_TOKEN"] = "your-ibm-quantum-token"
# os.environ["QISKIT_IBM_RUNTIME_MCP_INSTANCE"] = "your-instance-name"  # Optional but recommended for faster startup

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Verify configuration
print("Configuration status:")
print(f"  QISKIT_IBM_TOKEN: {'✓ Set' if os.getenv('QISKIT_IBM_TOKEN') else '✗ Not set'}")
print(f"  QISKIT_IBM_RUNTIME_MCP_INSTANCE: {'✓ Set (' + os.getenv('QISKIT_IBM_RUNTIME_MCP_INSTANCE') + ')' if os.getenv('QISKIT_IBM_RUNTIME_MCP_INSTANCE') else '✗ Not set (slower startup)'}")

### 3. Find Your IBM Quantum Instance (Optional but Recommended)

Setting `QISKIT_IBM_RUNTIME_MCP_INSTANCE` significantly speeds up startup (from ~30 seconds to ~2 seconds).

Run this cell to find your available instances:

In [ ]:
# Uncomment to find your available instances
# from qiskit_ibm_runtime import QiskitRuntimeService
# service = QiskitRuntimeService()
# print("Available instances:")
# for instance in service.instances():
#     print(f"  - {instance['name']} ({instance['plan']})")

## Imports and Configuration

In [ ]:
import warnings

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Suppress deprecation warning for create_react_agent
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    from langgraph.prebuilt import create_react_agent

## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [ ]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

In [ ]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
print("Using Anthropic Claude Sonnet")

In [ ]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

In [ ]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx
llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [ ]:
SYSTEM_PROMPT = """You are a helpful quantum computing assistant with access to IBM Quantum services
through the Qiskit IBM Runtime MCP server.

You can help users:
- Set up their IBM Quantum account (setup_ibm_quantum_account_tool)
- List available quantum backends (list_backends_tool)
- Find the least busy backend for running jobs (least_busy_backend_tool)
- Get detailed backend properties (get_backend_properties_tool)
- Get backend calibration data including T1, T2, error rates, and faulty qubits (get_backend_calibration_tool)
- List recent jobs (list_my_jobs_tool)
- Check job status (get_job_status_tool)
- Cancel jobs (cancel_job_tool)

Always provide clear explanations about quantum computing concepts when relevant.
When listing backends, highlight key properties like qubit count and operational status.
When showing calibration data, highlight faulty qubits/gates that users should avoid.
If an operation fails, explain the error and suggest possible solutions."""

## Create the MCP Client

This configures the connection to the qiskit-ibm-runtime-mcp-server:

In [ ]:
def get_mcp_client():
    """Create and return an MCP client configured for the Qiskit IBM Runtime server."""
    return MultiServerMCPClient(
        {
            "qiskit-ibm-runtime": {
                "transport": "stdio",
                "command": "qiskit-ibm-runtime-mcp-server",
                "args": [],
                "env": {
                    "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                    "QISKIT_IBM_RUNTIME_MCP_INSTANCE": os.getenv("QISKIT_IBM_RUNTIME_MCP_INSTANCE", ""),
                },
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Reuses the IBM Quantum service connection
- Makes tool calls much faster

In [ ]:
async def create_agent_with_session(session):
    """Create a LangGraph agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")
    
    # Create the ReAct agent
    agent = create_react_agent(llm, tools, prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [ ]:
from langchain_core.messages import HumanMessage

async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [ ]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

### Example 1: List Available Backends

Let's ask the agent to show us what quantum backends are available:

In [ ]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    
    response = await ask_agent(agent, "What quantum backends are available?")
    print(response)

### Example 2: Find the Least Busy Backend

When you want to run a job quickly, you want the backend with the shortest queue:

In [ ]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    
    response = await ask_agent(agent, "Which backend has the shortest queue right now?")
    print(response)

### Example 3: Get Backend Details

Let's get detailed information about a specific backend:

In [ ]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    
    response = await ask_agent(agent, "Tell me about the ibm_boston backend, including its calibration data")
    print(response)

### Example 4: Check Recent Jobs

See what quantum jobs you've submitted recently:

In [ ]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    
    response = await ask_agent(agent, "Show me my recent quantum jobs")
    print(response)

### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the agent:

In [ ]:
async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")
    
    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break
            
            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

## Custom Queries

Use this cell to ask the agent any question:

In [ ]:
# Enter your question here:
MY_QUESTION = "Compare the ibm_boston and ibm_sherbrooke backends"

async with mcp_client.session("qiskit-ibm-runtime") as session:
    agent = await create_agent_with_session(session)
    response = await ask_agent(agent, MY_QUESTION)
    print(response)

## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `setup_ibm_quantum_account_tool` | Set up IBM Quantum account with credentials |
| `list_backends_tool` | List all available quantum backends |
| `least_busy_backend_tool` | Find the least busy operational backend |
| `get_backend_properties_tool` | Get detailed properties of a specific backend |
| `get_backend_calibration_tool` | Get calibration data (T1, T2, error rates, faulty qubits) |
| `list_my_jobs_tool` | List user's recent quantum jobs |
| `get_job_status_tool` | Get status of a specific job |
| `cancel_job_tool` | Cancel a running or queued job |

## Troubleshooting

### Slow startup?
Set `QISKIT_IBM_RUNTIME_MCP_INSTANCE` environment variable to skip instance lookup.

### Authentication errors?
Verify your `QISKIT_IBM_TOKEN` is correct and your IBM Quantum account is active.

### MCP server not found?
Make sure `qiskit-ibm-runtime-mcp-server` is installed: `pip install qiskit-ibm-runtime-mcp-server`